# ЛР2. Код Хемминга и контрольные суммы
Акимова Алина БПМ-19-2

In [3]:
from typing import List
from math import log2, ceil
from random import randrange
from binascii import crc32

In [4]:
def __hamming_common(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0

    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]

            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)

        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1

    return errors

In [52]:
def hamming_encode(msg: str, mode: int = 8) -> str:
    """
    Encoding the message with Hamming code.

    :param msg: Message string to encode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    msg_b = msg.encode("utf-8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    bit_seq = []
    for byte in msg_b:  # get bytes to binary values; every bits store to sublist
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)     # length of result (bytes)
    bit_seq += [0] * (res_len * mode - len(bit_seq))    # filling zeros

    to_hamming = []

    for i in range(res_len):    # insert control bits into specified positions
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, True)   # process

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [53]:
def hamming_decode(msg: str, mode: int = 8):
    """
    Decoding the message with Hamming code.

    :param msg: Message string to decode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    res_len = len(msg) // (mode + s_num)    # length of result (bytes)
    code_len = mode + s_num     # length of one code sequence

    to_hamming = []

    for i in range(res_len):    # convert binary-like string to int-list
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, False)  # process

    for i in to_hamming:    # delete control bits
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):   # convert from binary-sring value to integer
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))

    # finally decode to a regular string
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [79]:
def noizer(msg: str, mode: int):
    """
    Generates an error in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result, cnt

In [78]:
def noizer3(msg: str, mode: int):
    """
    Generates up to 3 errors in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        result += "".join(map(str, to_noize))

    return result, cnt * 3

In [80]:
if __name__ == '__main__':
    MODE = 28  # длина слова с контрольными битами составляет 34 => значащих битов в слове 28
    msg = 'Одна из областей применения технологии одноранговых сетей — обмен файлами. Пользователи файлообменной сети выкладывают какие-либо файлы в папку общего доступа («расшаренную» от англ. share — делиться) на своём компьютере, содержимое которой доступно для скачивания другим пользователям. Какой-нибудь другой пользователь сети посылает запрос на поиск какого-либо файла. Программа ищет у клиентов сети файлы, соответствующие запросу, и показывает результат. После этого пользователь может скачать файлы у найденных источников. В современных файлообменных сетях информация загружается сразу из нескольких источников. Её целостность проверяется по контрольным суммам. Многие распространяемые в таких сетях файлы, не являющиеся свободными для распространения с юридической точки зрения, распространяются в них без разрешения правообладателей. Видеоиздательские и звукозаписывающие компании утверждают, что это приводит к значительной недополученной ими прибыли. Проблем им добавляет тот факт, что пресечь распространение файла в децентрализованной сети технически невозможно — для этого потребуется физически отключить от сети все устройства, на накопителях которых находится этот файл, а таких устройств (см. выше) может быть очень и очень много — в зависимости от популярности файла их число может достигать нескольких сотен тысяч. В последнее время издатели видеопродукции и звукозаписывающие компании стали подавать в суд на отдельных пользователей таких сетей, обвиняя их в незаконном распространении музыки и видео.Такие организации, как RIAA, дискредитируют пиринговые сети, публикуя в них фальшивые файлы. Это привело к потере популярности сети KaZaA в пользу eDonkey, имеющей более совершенную архитектуру. Несмотря на то, что в феврале 2006 прекратил работу самый популярный сервер сети eD2k — Razorback — и была прекращена разработка коммерческого клиента EDonkey2000, сама сеть ED2K продолжает функционировать, так как не завязана на конкретные серверы, и существует большое количество свободно распространяемых клиентских программ типа eMule и mlDonkey.'
    print(f'Сообщение:\n{msg}')
    checksum = crc32(b'{msg}')
    print(f'Контрольная сумма: {checksum}')

    # Первая отправка (без ошибок)
    print('-----------ПЕРВАЯ ОТПРАВКА-----------')
    enc_msg = hamming_encode(msg, MODE)
    print(f'Кодированное сообщение:\n{enc_msg}')
    dec_msg, err = hamming_decode(enc_msg, MODE)
    print(f'Раскодированное сообщение:\n{dec_msg}')
    new_checksum = crc32(b'{dec_msg}')
    print(
        f'Контрольная сумма: {new_checksum}, корректность: {new_checksum == checksum}')
    print(f'MSG: {msg == dec_msg}')

    # Вторая отправка (не более 1 ошибки на слово)
    print('-----------ВТОРАЯ ОТПРАВКА-----------')
    noize_msg, noize_err = noizer(enc_msg, MODE)
    print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
    dec_msg, err = hamming_decode(noize_msg, MODE)
    print(f'Раскодированное сообщение:\n{dec_msg}')
    new_checksum = crc32(b'{dec_msg}')
    print(
        f'Контрольная сумма: {new_checksum}, корректность: {new_checksum == checksum}, внесенные ошибки: {noize_err}')
    print(f'MSG: {msg == dec_msg}')

    # Третья отправка (3 ошибки на слово)
    print('-----------ТРЕТЬЯ ОТПРАВКА-----------')
    noize_msg, noize_err = noizer3(enc_msg, MODE)
    print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
    dec_msg, err = hamming_decode(noize_msg, MODE)
    print(f'Раскодированное сообщение:\n{dec_msg}')
    new_checksum = crc32(b'{dec_msg}')
    print(
        f'Контрольная сумма: {new_checksum}, корректность: {new_checksum == checksum}, количество обнаруженных ошибок: {err}, внесенные ошибки: {noize_err}')

Сообщение:
Одна из областей применения технологии одноранговых сетей — обмен файлами. Пользователи файлообменной сети выкладывают какие-либо файлы в папку общего доступа («расшаренную» от англ. share — делиться) на своём компьютере, содержимое которой доступно для скачивания другим пользователям. Какой-нибудь другой пользователь сети посылает запрос на поиск какого-либо файла. Программа ищет у клиентов сети файлы, соответствующие запросу, и показывает результат. После этого пользователь может скачать файлы у найденных источников. В современных файлообменных сетях информация загружается сразу из нескольких источников. Её целостность проверяется по контрольным суммам. Многие распространяемые в таких сетях файлы, не являющиеся свободными для распространения с юридической точки зрения, распространяются в них без разрешения правообладателей. Видеоиздательские и звукозаписывающие компании утверждают, что это приводит к значительной недополученной ими прибыли. Проблем им добавляет тот факт, ч